In [1]:
from hello.hello import *

In [2]:
app = open_hello('71.189.82.196:6')

In [7]:
alarms = [
    "%s_Low_Low%s",
    "%s_Low%s",
    "%s_High%s",
    "%s_High_High%s",
]

stuff = ("Agitation", "RPM"), ("DO", "%"), ("Filter_Oven", "C"),\
        ("Pressure", "psi"), ("Temp", "C"), ("Total_Gas", "LPM"),\
        ("pH", "")
def mk_alrm(grp, unit, templates):
    if unit:
        unit = "_(%s)" % unit
    return [t % (grp, unit) for t in templates]

alrms = [mk_alrm(g, u, alarms) for g, u in stuff]

In [ ]:
cfg = app.getConfig()['Process_Alarms']
assert all(k in cfg for g in alrms for k in g)

In [78]:
from time import sleep


def test_alarm(app, alrm, pv, offset, cfg):
    app.login()
    app.setconfig("Process_Alarms", alrm, pv + offset)
    yield
    app.login()
    app.setconfig("Process_Alarms", alrm, cfg[alrm])

def test_alarm_group(app, alrm_grp, unit, mv_func, cfg):
    alrms = mk_alrm(alrm_grp, unit, alarms)
    offsets = (20, 10, -10, -20)
    exp_code = 90, 40, 50, 100
    tests = zip(alrms, offsets, exp_code)
    
    for a, o, c in tests:
        t = test_alarm(app, a, mv_func()['pv'], o, cfg)
        while mv_func()['error'] != 0: sleep(0.1)
        num_alarms = app.getUnAckCount()
        next(t)
        while mv_func()['error'] == 0: sleep(0.1)
        while app.getUnAckCount() <= num_alarms: sleep(0.1)
        e = mv_func()['error']
        res = 'pass' if c == e else 'fail'
        print(a, 'error:', e, "...", res)
        next(t, None)

def test_ag(app, cfg):
    for a in mk_alrm('Agitation', 'RPM', alarms):
        app.login()
        print("\rSetting config:", a, cfg[a], "                ",end="")
        app.setconfig('Process_Alarms', a, cfg[a])
    print("\r                                  ")
    f = lambda: app.gpmv()['agitation']
    test_alarm_group(app, 'Agitation', 'RPM', f, cfg)

def test_grp(app, grp, unit, mv_grp, cfg):
    for a in mk_alrm(grp, unit, alarms):
        app.login()
        print("\rSetting config:", a, cfg[a], "                ",end="")
        app.setconfig('Process_Alarms', a, cfg[a])
    print("\r                                  \r", end="")
    f = lambda: app.gpmv()[mv_grp]
    test_alarm_group(app, grp, unit, f, cfg)

In [79]:
groups = (
     ('Agitation', 'RPM', 'agitation'),
     ('DO', '%', 'do'),
     ('Filter_Oven', 'C', 'condenser'),
     ('Pressure', 'psi', 'pressure'),
     ('Temp', 'C', 'temperature'),
     ('Total_Gas', 'LPM', 'maingas'),
     ('pH', '', 'ph'),
     ('Level', 'L', 'level')
)

In [80]:
def run_test(app, groups, cfg):
    for g, u, mvg in groups:
        print("Testing", g)
        test_grp(app, g, u, mvg, cfg)

In [81]:
run_test(app, groups, cfg)

Testing Agitation
Agitation_Low_Low_(RPM) error: 90 ... pass
Agitation_Low_(RPM) error: 40 ... pass
Agitation_High_(RPM) error: 50 ... pass
Agitation_High_High_(RPM) error: 100 ... pass
Testing DO
DO_Low_Low_(%) error: 90 ... pass
DO_Low_(%) error: 40 ... pass
DO_High_(%) error: 50 ... pass
DO_High_High_(%) error: 100 ... pass
Testing Filter_Oven
Filter_Oven_Low_Low_(C) error: 90 ... pass
Filter_Oven_Low_(C) error: 40 ... pass
Filter_Oven_High_(C) error: 50 ... pass
Filter_Oven_High_High_(C) error: 100 ... pass
Testing Pressure
Pressure_Low_Low_(psi) error: 90 ... pass
Pressure_Low_(psi) error: 40 ... pass
Pressure_High_(psi) error: 50 ... pass
Pressure_High_High_(psi) error: 100 ... pass
Testing Temp
Temp_Low_Low_(C) error: 90 ... pass
Temp_Low_(C) error: 40 ... pass
Temp_High_(C) error: 50 ... pass
Temp_High_High_(C) error: 100 ... pass
Testing Total_Gas
Total_Gas_Low_Low_(LPM) error: 90 ... pass
Total_Gas_Low_(LPM) error: 40 ... pass
Total_Gas_High_(LPM) error: 50 ... pass
Total_Gas